In [ ]:
import pymongo


mongo 실행시켜서 port 번호 받기

In [ ]:
'''
MongoDB 연결 -> MongoClient('ip주소',포트번호) 객체 생성
                or
                MongoClient()
                 ㄴ -> 자동으로 localhost:27017접속
'''
conn = pymongo.MongoClient()


In [ ]:
'''
home_test db 생성

변수 = 연결객체(conn).db이름
'''
db = conn.db_home2
print(db)

In [ ]:
# 컬렉션 생성 형식) db.컬렉션 이름
home_col2 = db.home_col2
print(home_col2)

In [ ]:
post = {
            "author":"global", 
            "text":"global text", 
            "tags":["python","pymongo",'mongodb']
       }
post_id = home_col2.insert_one(post).inserted_id
post_id

In [ ]:
home_col.insert_many(
[{"author":"Tom Cruise", "age":55},{"author":"Tom", "age":32}]
    
)

In [ ]:
#document 개수 확인
all_docs = home_col.count_documents({})
all_docs

In [ ]:
data = []
data.append({'title':'avatar','castings':['zoe','sam','sigourney']})
data.append( {
        'title':'실미도',
        'castings':['설경구', '안성기'],
        'datetime':
            {
                'year':'2003',
                'month':3,
                'val':
                    {
                        'a':
                            {
                                'b':1
                            }
                    }
            }
    })
data.append({'name':'arron', 'age':23})
data.append({'name':'bob', 'age':32})
data.append({'name':'cathy', 'age':25})
data.append({'name':'peter', 'age':27})
data.append({'name':'erick', 'age':28})
data.append({'name':'davison', 'age':36})
data.append({'name':'lee'})

home_col.insert_many(data)

In [ ]:
home_col.find_one()

In [ ]:
all_docs=home_col.find({})
for i in all_docs:
    print(i)

In [ ]:
'''

컬렉션.update_one -> 조건에 맞아도 하나만 수정함, 조건없으면 첫 검색 수정
many -> 조건에 맞는 모든걸 수정

update로 수정,추가 둘다 가능
'''
import pprint as p
home_col.update_one({'author':'global'},{'$set':{'text':'globalhome'}})
for i in home_col.find({'author':'global'}):
    print(i)

In [ ]:
home_col.update_many({'author':'global'},{'$set':{'test':'testing'}})
for i in home_col.find({'author':'global'}):
    print(i)

In [ ]:
'''
document 삭제 수정과 동일하게 one 조건에 맞아도 하나만, 삭제 조건없으면 첫검색 삭제
many 조건에 맞는 모든 key 삭제
'''

In [ ]:
home_col.delete_one({'author':'Tom'})
for i in home_col.find({}):
    print(i)

In [ ]:
home_col.insert_one(
{"author":"Tom", "age":32}
    
)

In [ ]:
for i in home_col.find({}):
    print(i)

In [ ]:
# home_col.delete_many({'author':{'$in':'Tom'}}) 
    #$in needs an array, full error: {'index': 0, 'code': 2, 'errmsg': '$in needs an array'}
#-> [] 둘러싸줌
home_col.delete_many({'author':{'$in':['Tom']}}) # -> 톰크루즈 삭제안됨
for i in home_col.find({}):
    print(i)

In [ ]:
home_col.delete_many({'author':{'$in':['Tom','Tom Cruise']}}) 
    # -> RDBMS in과는 달리 아예 똑같아야 인식함
for i in home_col.find({}):
    print(i)

In [ ]:
'''
라이브러리

BeautifulSoup
html 데이터를 분석할 수 있는 형태로 만들어주는 클래스
'''
from bs4 import BeautifulSoup
import requests
import pymongo
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Mongodb과 python 연결 객체생생
conn = pymongo.MongoClient()
# DB 생성 이름 cine21_home
act_db = conn.cine21_home

In [ ]:
# 컬렉션 생성
act_col = act_db.act_col

In [ ]:
# 크롤링 주소 request하기
'''
-> 6개월 인기 배우 조회 버튼 클릭시 나옴
-> 해당 클릭이 웹페이지가 안바뀌고 data가 바껴서 post방식?
section: 
actor
period_start: 
2022-06
gender: 
all
page: 
1
'''
url = 'http://www.cine21.com/rank/person/content'
# url에서 /content 부분은 관리자모드 네트워크에서 정렬기준 클릭시 확인되는 주소임
# http://www.cine21.com/rank/person 만하면 인식 못함
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2022-06'
post_data['gender']='all'
post_data['page'] = 1

response = requests.post(url,post_data)

In [ ]:
'''
BeautifilSoup로 parsing (html 데이터를 분석할 수 있게 변환해줌)
'''
#                                            ┌ parser(파서(문장의 구조 분석·오류 점검 프로그램).) 이름
html_data = BeautifulSoup(response.content,'html.parser')
#                             ㄴ url,post_data에 대한 컨텐츠     

In [ ]:
#해당 페이지에 해당하는 배우 이름만 출려해보기
actors_list = html_data.select('li.people_li div.name')
for i in actors_list:
    print(i)
    print(i.text)
print('-'*70)

In [ ]:
import re
for i in actors_list:
    print(re.sub('\W','',i.text))

In [ ]:
for i in actors_list:
    print(re.sub('[유해진]','',i.text))

In [ ]:
for i in actors_list:
    print(re.sub('유|현|박|임|정|변|안','',i.text))

In [ ]:
'''
  [0-9] (\d)          :  모든 숫자와 매칭됨
  [^0-9] (\D)         :  숫자가 아닌 것과 매칭됨
  [\t\n\r\f\v] (\s)   :  whitespace 문자와 매칭됨
  [^\t\n\r\f\v] (\S)  :  whitespace 문자가 아닌 것과 매칭됨
  [a-zA-Z0-9]  (\w)   :  문자 + 숫자와 매칭됨
  [^a-zA-Z0-9] (\W)   :  문자 + 숫자가 아닌 것과 매칭됨
'''

for i in actors_list:
    print(re.sub('[\(\d편\)]','',i.text))

In [ ]:
for i in actors_list:
    print(re.sub('\(\w*\)','',i.text))

In [ ]:
for i in actors_list:
    print(re.sub('\(\w+\)','',i.text))

In [ ]:
for i in actors_list:
    print(re.sub('\(\w?\)','',i.text))

In [ ]:
actors_list = html_data.select('li.people_li div.name')
for i in actors_list:
    print(i)
    print(i.text)
print('-'*70)

In [ ]:
'''
배우 상세 정보 가져오기
유해진 클릭시 http://www.cine21.com/db/person/info/?person_id=15757 로 이동함
여기서 해당 앵커 html 위치
li.people_li div.name a의 href 값 db/person/info/?person_id=15757
'''
print(html_data.select('li.people_li div.name a'))
print()
print(html_data.select_one('li.people_li div.name a'))
print()
print(actors_list[0].select_one('a'))
print(html_data.select_one('li.people_li div.name a').attrs['href'])
#  ㄴ a 태그에서 href 값을 추출함

#http://www.cine21.com + /db/person/info/?person_id=15757 해주면 해당 배우의 상세페이지로 이동함
print('http://www.cine21.com'+html_data.select_one('li.people_li div.name a').attrs['href'])

In [ ]:

'''
페이지의 배우별 상세페이지 url
'''
actors_detail_url= []
idx = 0
for i in actors_list:
    actors_detail_url.append('http://www.cine21.com' + i.select_one('a').attrs['href'])
    
    print(actors_detail_url[idx])
    idx+=1

In [ ]:
'''
상세페이질 이동 후 가지고 올 데이터
'''
actors_detail_data=[]
for idx,i in enumerate(actors_detail_url):
    response = requests.get(i) 
        #--> request get방식이라 웹페이지 이동하고 해당 html가져오는듯?
    actors_detail_html_data = BeautifulSoup(response.content,'html.parser')
    actors_detail_data.append(actors_detail_html_data.select('ul.default_info li'))
actors_detail_data

In [ ]:
actors_detail_list=[]
#for 문 하나에 상세페이지로 이동후 기본정보란의 데이터 dict에 저장
for i in actors_list:
    actors_detail_url= 'http://www.cine21.com' + i.select_one('a').attrs['href']
    response = requests.get(actors_detail_url) 
    actors_detail_html_data = BeautifulSoup(response.content,'html.parser')
    actors_detail_dict={}
    for j in actors_detail_html_data.select('ul.default_info li'):
        # -> 태그 li,span중 span만 잡아서 text?
        key = j.select_one('span').text 
        value = (str(j).split('</span>')[1].split('</li>')[0])
        actors_detail_dict[key] = value
    actors_detail_list.append(actors_detail_dict)
actors_detail_list

In [ ]:
'''
6) 각 배우의 이름, 흥해지수, 출연 가져오기
'''
actors_hits = html_data.select('ul.num_info li strong')
actors_movies = html_data.select('ul.mov_list')
actors_info_list=[]


for idx,i in enumerate(actors_list):
    movie_list=[]
    
    a = actors_movies[idx].select('li a span')
    for j in a:
        movie_list.append(j.text)
    
    actors_info_dict=dict()   
    
    actors_info_dict['배우이름'] = re.sub('\(\w*\)','',i.text)
    actors_info_dict['흥행지수'] = actors_hits[idx].text
    actors_info_dict['출연영화'] = movie_list
    
    actors_info_list.append(actors_info_dict)
actors_info_list

In [ ]:
import re
test = []

a= actors_movies[0].select('li a span') # -> for 껍질 까줘야지 text로 접근할 수있음
for i in a:
    print(i.text)

In [ ]:
'''
한 코드로 합쳐기
'''


In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import warnings
warnings.filterwarnings('ignore')


conn = pymongo.MongoClient()
act_db = conn.cine21_home
act_col = act_db.act_col

url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2022-06'
post_data['gender']='all'
post_data['page'] = 1

response = requests.post(url,post_data)
html_data = BeautifulSoup(response.content,'html.parser')

actors_hits = html_data.select('ul.num_info li strong')
actors_movies = html_data.select('ul.mov_list')
actors_info_list=[]

for idx,i in enumerate(actors_list):
    movie_list=[]
    actors_info_dict=dict()   
    
    a = actors_movies[idx].select('li a span')
    for j in a:
        movie_list.append(j.text)
    
    
    actors_info_dict['배우이름'] = re.sub('\(\w*\)','',i.text)
    actors_info_dict['흥행지수'] = actors_hits[idx].text
    actors_info_dict['출연영화'] = movie_list
    
    
    
    actors_detail_url= 'http://www.cine21.com' + i.select_one('a').attrs['href']
    response = requests.get(actors_detail_url) 
    actors_detail_html_data = BeautifulSoup(response.content,'html.parser')
    for j in actors_detail_html_data.select('ul.default_info li'):
        # -> 태그 li,span중 span만 잡아서 text?
        key = j.select_one('span').text 
        value = (str(j).split('</span>')[1].split('</li>')[0])
        actors_info_dict[key] = value
    actors_info_list.append(actors_info_dict)


In [ ]:
actors_info_list

In [ ]:
'''
post_data 의 값을 변경하여 다른페이지의 데이터도 가져오기
'''

In [ ]:
from bs4 import BeautifulSoup
import requests
import pymongo
import re
import warnings
warnings.filterwarnings('ignore')


conn = pymongo.MongoClient()
act_db = conn.cine21_home
act_col = act_db.act_col

url = 'http://www.cine21.com/rank/person/content'
post_data = dict()
post_data['section']='actor'
post_data['period_start']='2022-06'
post_data['gender']='all'
actors_info_list=[]


for n in range(1,21):
    post_data['page'] = n
    response = requests.post(url,post_data)
    html_data = BeautifulSoup(response.content,'html.parser')
    actors_list = html_data.select('li.people_li div.name')
    actors_hits = html_data.select('ul.num_info li strong')
    actors_movies = html_data.select('ul.mov_list')
    
    
    for idx,i in enumerate(actors_list):
        movie_list=[]
        actors_info_dict=dict()   

        a = actors_movies[idx].select('li a span')
        for j in a:
            movie_list.append(j.text)


        actors_info_dict['배우이름'] = re.sub('\(\w*\)','',i.text)
        actors_info_dict['흥행지수'] = int(actors_hits[idx].text.replace(',',''))
                #  ㄴ 아 숫자로 안바꿔져서 sort하면 이상하게나옴.......
        actors_info_dict['출연영화'] = movie_list



        actors_detail_url= 'http://www.cine21.com' + i.select_one('a').attrs['href']
        response = requests.get(actors_detail_url) 
        actors_detail_html_data = BeautifulSoup(response.content,'html.parser')
        for j in actors_detail_html_data.select('ul.default_info li'):
            # -> 태그 li,span중 span만 잡아서 text?
            key = j.select_one('span').text 
            value = (str(j).split('</span>')[1].split('</li>')[0])
            actors_info_dict[key] = value
        actors_info_list.append(actors_info_dict)


In [ ]:
actors_info_list

In [ ]:
act_col.insert_many(actors_info_list)

In [ ]:
'''
key 이름 변경하기
update , $rename 사용
update_many({},{$rename:{{key},{변경key name}}})
'''
act_col.update_many({},{'$rename':{'다른 이름': '다른이름'}})

In [ ]:
import pprint as p
for i in act_col.find():
    p.pprint(i)

In [ ]:
'''
저장한 데이터 검색방법
-sort()                         ┌ default : 오름차순(pymongo.ASCENDING)
find({}).sort('정렬기준(key)',정렬방법)

-$exists
find({key:'{'$exists':True}'})

-$nor
find({'$nor':[{조건},{조건}...]})

-$or
find({'$or':[{'key':조건},{'key':조건}...]})

-$in , $nin -> RDBMS 처럼포함은 안쳐줌
find({'key':{$in:{[value,value...]}}})

-skip(n) n개만큼 건너띄고 출력
find({}).skip(n)

-$all
find({key:{$all:[value,value])

-리스트 index로 조회
find({key.index:value})

-리스트 사이즈로 조회 $size
find({key:{$size:숫자}})
'''

In [ ]:
a = act_col.find({}).sort('흥행지수')
for i in a:
    p.pprint(i)

In [ ]:
a = act_col.find({}).sort('흥행지수',pymongo.DESCENDING)
for i in a:
    p.pprint(i)

In [ ]:
for i in act_col.find({'출연영화':['올빼미']}): #무조건 똑같아야 출력
    p.pprint(i)

In [ ]:
for i in act_col.find({'출연영화':{'$all':['올빼미','공조']}}): # -> 순서는 상관없음
    p.pprint(i)

In [ ]:
for i in act_col.find({'출연영화.1':'올빼미'}): # -> 두번째 영화가 올빼미인 배우
    p.pprint(i)

In [ ]:
for i in act_col.find({'출연영화':{'$size':3}}): # -> 두번째 영화가 올빼미인 배우
    p.pprint(i)

In [256]:
conn = pymongo.MongoClient()
ex_db = conn.ex_db
ex_col = ex_db.ex_col

In [257]:
ex_col.insert_many([
    {'results':[82, 85, 88]},  
    {'results':[75, 88, 91]},
    {'results':[75, 88, 89]},
    {'results':[85, 88, 92]}
])

In [ ]:
'''
eleMatch
    ㄴ 적어도 하나 이상의 리스트 요소가 여러 조건을 동시에 만족하는 경우 출력
'''

In [260]:
for i in ex_col.find({'results':{'$gte':90,'$lt':85}}):
    print(i)

{'_id': ObjectId('63aad197ea0b1f6ecd117983'), 'results': [75, 88, 91]}


In [263]:
for i in ex_col.find({'results':{'$elemMatch':{'$gte':90,'$lt':85}}}):
    print(i)

{'_id': ObjectId('63aad197ea0b1f6ecd117982'), 'results': [82, 85, 88]}
{'_id': ObjectId('63aad197ea0b1f6ecd117983'), 'results': [75, 88, 91]}
{'_id': ObjectId('63aad197ea0b1f6ecd117984'), 'results': [75, 88, 89]}
{'_id': ObjectId('63aad197ea0b1f6ecd117985'), 'results': [85, 88, 92]}


In [265]:
for i in ex_col.find({'%or':[{'results':{'$gte':90}},{'results':{'lt':85}}]}):
    print(i)

In [267]:
for i in ex_col.find({'%or':[{'results':{'$gte':75}},{'results':{'lt':90}}]}):
    print(i)